# Парсинг информации о странах и их рейтинга

## Подключение нужных библиотек

In [1]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from tqdm import tqdm

##  Парсинг стран и их рейтинга в Happiness Score

In [2]:
def get_happiness_data(url = "https://en.wikipedia.org/wiki/World_Happiness_Report"):
    """
    Собирает информацию о странах и их рейтинге в Happiness Score с Википедии
    Возвращает: DataFrame
    """

    try:
        happiness_df = pd.read_html(url)[15]
    except Exception as e:
        print(f"Error reading HTML tables: {e}")
        return pd.DataFrame()

    happiness_df.rename(columns={'Overall rank': 'Happiness rank',
                                 'Country or region': 'Country',
                                 'Life evaluation': 'Happiness score'},
                        inplace=True)
    return happiness_df

In [3]:
countries_happiness_df = get_happiness_data()

os.makedirs('data', exist_ok=True)
countries_happiness_df.to_csv('data/countries_happiness.csv', index=False)

## Парсинг различных показателей стран с World Bank Open Data

In [4]:
BASE_URL = "https://data.worldbank.org"

def get_country_links(base_url=BASE_URL):
    """
    Достает ссылки на все страны из подкаталога country
    Возвращает массив пар: (Название страны, Ссылка на страну)
    """
    url = urljoin(base_url, "country")
    print(f"Ищем ссылки на страны на странице: {url}")
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, "html.parser")
    country_links = []

    # Find all <a> tags whose href begins with "/country/"
    for a in soup.find_all("a", href=True):
        href = a['href']
        if href.startswith("/country/"):
            country_name = a.get_text(strip=True)
            full_url = urljoin(base_url, href)
            # Avoid duplicates
            if country_name and full_url not in [link for (_, link) in country_links]:
                country_links.append((country_name, full_url))

    print(f"Найдено {len(country_links)} стран.")
    return country_links

In [19]:
def get_country_indicators(country_url):
    """
    Принимает URL страницы страны (например, "https://data.worldbank.org/country/finland?view=chart")
    и возвращает:
      - словарь с данными индикаторов: {название индикатора: значение (float или None)},
      - словарь с метаданными: {название индикатора: ссылка на индикатор}.
    """

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(country_url)

    wait = WebDriverWait(driver, 5)
    try:
        # Ищем вкладку "By Theme" по XPath (элемент <li> с классом react-tabs__tab, внутри которого есть <span> с
        # нужным текстом)
        by_theme_tab = wait.until(
            ec.element_to_be_clickable(
                (By.XPATH, "//li[contains(@class, 'react-tabs__tab') and .//span[contains(text(), 'By Theme')]]")
            )
        )
        # Кликаем на нужную вкладку
        driver.execute_script("arguments[0].click();", by_theme_tab)

        # Ждем появления контента вкладки
        wait.until(ec.presence_of_element_located((By.CSS_SELECTOR, ".indicator-item")))
        time.sleep(2)  # небольшая задержка
        html = driver.page_source
        # print(f"Вкладка 'By Theme' успешно открыта для {country_url}")
    except Exception as e:
        # print(f"Не удалось найти или кликнуть по вкладке 'By Theme' для {country_url}: {e}")
        driver.quit()
        return {}, {}
    finally:
        driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    indicators_info = soup.find_all(class_='indicator-item__inner')

    indicator_values = {}
    indicator_links = {}

    for indicator_info in indicators_info:
        headline_elem = indicator_info.find(class_='indicator-item__headline-mobile')
        if headline_elem:
            a_tag = headline_elem.find('a')
            if a_tag:
                indicator_name = a_tag.get_text(strip=True)
                indicator_link = a_tag.get('href')
                indicator_link = indicator_link.split('?locations=')[0]
                indicator_links[indicator_name] = indicator_link

                data_info = indicator_info.find(class_='indicator-item__data-info')
                if data_info:
                    span_tag = data_info.find('span')
                    if span_tag:
                        try:
                            # Убираем запятые, пробуем преобразовать текст в число
                            value = float(span_tag.get_text(strip=True).replace(',', ''))
                        except Exception as ex:
                            print(f"Ошибка преобразования значения для '{indicator_name}': {ex}")
                            value = None
                    else:
                        value = None
                else:
                    value = None
                indicator_values[indicator_name] = value

    return indicator_values, indicator_links

### Собираем информацию по всем индикаторам для всех стран

In [16]:
countries = get_country_links()

all_country_data = []  # список словарей: каждый словарь содержит данные для одной страны
metadata_dict = {}     # метаданные (предполагается, что структура индикаторов одинакова для всех стран)
collected_countries = set() # названия стран, информация про которые уже собрана, чтобы пропускать их при повторном запуске

Ищем ссылки на страны на странице: https://data.worldbank.org/country
Найдено 217 стран.


In [20]:
for country_name, country_url in (pbar := tqdm(countries)):
    pbar.set_description(f"Собираем информацию о стране {country_name}")

    if country_name in collected_countries:
        continue

    attempt = 0
    wait_time = 10
    # Первая попытка получения данных
    indicators, metadata = get_country_indicators(country_url)
    # Если список индикаторов пуст, делаем повторные попытки до 10 раз
    while len(indicators) == 0 and attempt < 10:
        attempt += 1
        pbar.write(f"Попытка {attempt} для {country_name} неуспешна. Ожидание {wait_time} секунд перед повторной попыткой...")
        time.sleep(wait_time)
        wait_time += 10
        indicators, metadata = get_country_indicators(country_url)

    if len(indicators) == 0:
        pbar.write(f"Не удалось получить данные для {country_name} после 10 попыток.")
    else:
        collected_countries.add(country_name)
    # Приводим ссылки индикаторов к полному виду
    metadata = {indicator_name: urljoin(BASE_URL, metadata[indicator_name])
                for indicator_name in metadata.keys()}

    # Формируем словарь с данными: имя страны + индикаторы
    data_entry = {"Country": country_name}
    data_entry.update(indicators)
    all_country_data.append(data_entry)

    # Обновляем общий словарь метаданных
    metadata_dict.update(metadata)

    time.sleep(2)  # пауза между запросами к разным странам

Собираем информацию о стране Central African Republic:  17%|█▋        | 37/217 [05:37<52:15, 17.42s/it]

Попытка 1 для Central African Republic неуспешна. Ожидание 10 секунд перед повторной попыткой...


Собираем информацию о стране Channel Islands:  18%|█▊        | 39/217 [06:48<1:12:36, 24.47s/it]         

Попытка 1 для Channel Islands неуспешна. Ожидание 10 секунд перед повторной попыткой...


Собираем информацию о стране Ecuador:  26%|██▋       | 57/217 [13:51<49:58, 18.74s/it]           

...

Собираем информацию о стране Togo:  89%|████████▉ | 194/217 [1:10:38<07:36, 19.86s/it]                          

Попытка 1 для Togo неуспешна. Ожидание 10 секунд перед повторной попыткой...


Собираем информацию о стране Zambia:  99%|█████████▉| 215/217 [1:19:27<00:34, 17.13s/it]                  

Попытка 1 для Zambia неуспешна. Ожидание 10 секунд перед повторной попыткой...


Собираем информацию о стране Zimbabwe: 100%|██████████| 217/217 [1:20:34<00:00, 22.28s/it]


### Объединяем всю информацию в один DataFrame и сохраняем

In [21]:
os.makedirs('data', exist_ok=True)

# Создаем DataFrame, где индекс – название страны, а столбцы – названия индикаторов
df = pd.DataFrame(all_country_data)
df.set_index("Country", inplace=True)
df.to_csv("data/countries_indicators.csv", encoding="utf-8")
print("Сохранены данные по индикаторам для стран в файле 'data/country_indicators.csv'.")

# Сохраняем метаданные в отдельный CSV: для каждого индикатора его ссылка
meta_df = pd.DataFrame(list(metadata_dict.items()), columns=["Indicator", "Link"])
meta_df.to_csv("data/indicators_metadata.csv", index=False, encoding="utf-8")
print("Сохранены метаданные индикаторов в файле 'data/indicators_metadata.csv'.")

Сохранены данные по индикаторам для стран в файле 'data/country_indicators.csv'.
Сохранены метаданные индикаторов в файле 'data/indicators_metadata.csv'.


## Собираем данные о странах с помощью REST Countries API

### Получаем всю информацию о всех странах

In [ ]:
url = "https://restcountries.com/v3.1/all"
response = requests.get(url)

if response.status_code == 200:
    countries = response.json()
else:
    print("Error:", response.status_code)
    exit()

### Выбираем нужную информацию и сохраняем

In [37]:
country_list = []
for country in countries:
    name = country.get("name", {}).get("common", None)
    region = country.get("region", None)
    languages = list(country.get("languages", {}).values()) if country.get("languages") else None
    main_language = languages[0] if languages else None
    independent = country.get("independent", None)
    un_member = country.get("unMember", None)
    currencies = country.get("currencies", {})
    main_currency = list(currencies.keys())[0] if currencies else None
    population = country.get("population", None)
    area = country.get("area", None)
    car_side = country.get("car", {}).get("side", None)

    country_list.append({
        "Country": name,
        "Region": region,
        "Main Language": main_language,
        "Independent": independent,
        "UN Member": un_member,
        "Main Currency": main_currency,
        "Population": population,
        "Area": area,
        "Car Side": car_side
    })

# Сохраняем в CSV
df = pd.DataFrame(country_list)
df.to_csv("data/countries_base_info.csv", index=False)

print("Сохраняем данные для стран в 'data/countries_base_info.csv'")

Сохраняем данные для стран в 'data/countries_base_info.csv'
